In [2]:
import ipydeps
ipydeps.pip(['IPython',
             'tensorflow',
             'requests',
             'tqdm',
             'pandas',
             'joblib',
             'scikit-learn'],verbose=False)
import IPython
import tensorflow as tf
import pandas as pd
import json
import os
import math
from datetime import datetime
import joblib
from sklearn.preprocessing import QuantileTransformer
# importing classes from other files that assist with the downloading and
#  feature engineering of Baltimore's arcgis data
from Baltimore_Data_Modules.BaltimoreDataExtraction import BmoreDataExtraction
from Baltimore_Data_Modules.Bmore_FeatureEngineering import BmoreCrimeFeatureEngineering
import Baltimore_Data_Modules.Model_Utils as Model_Utils

def days_hours_minutes(td):
    # function that is used for converting time into a readable format
    return td.days, td.seconds//3600, (td.seconds//60)%60

# Initializing Baltimore Data Extraction class
- The API link dictionary organizes each data set by type of data set

In [ ]:
from Baltimore_Data_Modules import BaltimoreDataExtraction

Bmore_API_Crime_link_dict = {'FEAT_COLLECT':
                             {#'911 Calls for Service 2022 Through Present':'https://services1.arcgis.com/UWYHeuuJISiGmgXx/ArcGIS/rest/services/911_CallsForService_PreviousYear_Present/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson',
                              #'BPD Arrests':'https://egis.baltimorecity.gov/egis/rest/services/GeoSpatialized_Tables/Arrest/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson',
                              #'Parking and Moving Citations':'https://services1.arcgis.com/UWYHeuuJISiGmgXx/arcgis/rest/services/Parking_and_Moving_Citations__view/FeatureServer/2/query?outFields=*&where=1%3D1&f=geojson',
                              #'Gun Offenders Registry':'https://egis.baltimorecity.gov/egis/rest/services/GeoSpatialized_Tables/GunOffenders/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson',
                              'Part 1 Crime Data':'https://services1.arcgis.com/UWYHeuuJISiGmgXx/arcgis/rest/services/Part1_Crime_Beta/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson'},
                             'CSA_DATA':
                             {'Percent of Adult Population on Parole/Probation - Community Statistical Area':'https://services1.arcgis.com/mVFRs7NF4iFitgbY/arcgis/rest/services/Prbprl/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson',
                              'Property Crime Rate per 1,000 Residents - Community Statistical Area':'https://services1.arcgis.com/mVFRs7NF4iFitgbY/arcgis/rest/services/Prop/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson',
                              'Violent Crime Rate per 1,000 Residents - Community Statistical Area':'https://services1.arcgis.com/mVFRs7NF4iFitgbY/arcgis/rest/services/Viol/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson',
                              'Part 1 Crime Rate per 1,000 Residents - Community Statistical Area':'https://services1.arcgis.com/mVFRs7NF4iFitgbY/arcgis/rest/services/Crime/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson',
                              'Number of Gun-Related Homicides per 1,000 Residents - Community Statistical Area':'https://services1.arcgis.com/mVFRs7NF4iFitgbY/arcgis/rest/services/Gunhom/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson',
                              'Number of Arrests per 1,000 Residents - Community Statistical Area':'https://services1.arcgis.com/mVFRs7NF4iFitgbY/arcgis/rest/services/Arrests/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson',
                              'Domestic Violence Calls For Service per 1,000 Residents - Community Statistical Area':'https://services1.arcgis.com/mVFRs7NF4iFitgbY/arcgis/rest/services/Juvarr/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson',
                              'Juvenile Arrest Rate per 1,000 Juveniles - Community Statistical Area':'https://services1.arcgis.com/mVFRs7NF4iFitgbY/arcgis/rest/services/Juvarr/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson',
                              'Juvenile Arrest Rate for Violent Offenses per 1,000 Juveniles - Community Statistical Area':'https://services1.arcgis.com/mVFRs7NF4iFitgbY/arcgis/rest/services/Juvviol/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson',
                              'Juvenile Arrest Rate for Drug-Related Offenses per 1,000 Juveniles - Community Statistical Area':'https://services1.arcgis.com/mVFRs7NF4iFitgbY/arcgis/rest/services/Juvdrug/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson',
                              'Number of Narcotics Calls for Service per 1,000 Residents - Community Statistical Area':'https://services1.arcgis.com/mVFRs7NF4iFitgbY/arcgis/rest/services/Narc/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson'},
                             'CITY_DATA':
                             {'Property Crime Rate per 1,000 Residents - City':'https://services1.arcgis.com/mVFRs7NF4iFitgbY/arcgis/rest/services/Prop/FeatureServer/1/query?outFields=*&where=1%3D1&f=geojson',
                             'Violent Crime Rate per 1,000 Residents - City':'https://services1.arcgis.com/mVFRs7NF4iFitgbY/arcgis/rest/services/Viol/FeatureServer/1/query?outFields=*&where=1%3D1&f=geojson',
                             'Part 1 Crime Rate per 1,000 Residents - City':'https://services1.arcgis.com/mVFRs7NF4iFitgbY/arcgis/rest/services/Crime/FeatureServer/1/query?outFields=*&where=1%3D1&f=geojson',
                             'Number of Gun-Related Homicides per 1,000 Residents - City':'https://services1.arcgis.com/mVFRs7NF4iFitgbY/arcgis/rest/services/Gunhom/FeatureServer/1/query?outFields=*&where=1%3D1&f=geojson',
                             'Number of Narcotics Calls for Service per 1,000 Residents - City':'https://services1.arcgis.com/mVFRs7NF4iFitgbY/arcgis/rest/services/Narc/FeatureServer/1/query?outFields=*&where=1%3D1&f=geojson',
                             'Number of Common Assault Calls for Service per 1,000 Residents - City':'https://services1.arcgis.com/mVFRs7NF4iFitgbY/arcgis/rest/services/Caslt/FeatureServer/1/query?outFields=*&where=1%3D1&f=geojson',
                             'Number of Common Assault Calls for Service per 1,000 Residents - City':'https://services1.arcgis.com/mVFRs7NF4iFitgbY/arcgis/rest/services/Caslt/FeatureServer/1/query?outFields=*&where=1%3D1&f=geojson'}
                            }

with open("Bmore_API_Crime_link.json", "w") as outfile:
    json.dump(Bmore_API_Crime_link_dict, outfile, indent=4)


bde = BaltimoreDataExtraction.BmoreDataExtraction(Bmore_API_Crime_link_dict)
bde

# Downloading latest data set by type of data
- next 3 cells

In [ ]:
FEAT_COLLECT_Path_dict = bde.DownloadDataType(data_type_str='FEAT_COLLECT',includeAll=True)

def Nested_featCol_Union(nestedPath_dict,chosenKey):
    oneDataset_dict = nestedPath_dict[chosenKey]
    oneDatasetDF_dict = bde.FeatureCollectionToDF(oneDataset_dict)
    return {chosenKey:pd.concat(list(oneDatasetDF_dict.values()))}

FEAT_COLLECT_DF_dict = Nested_featCol_Union(FEAT_COLLECT_Path_dict,'Part 1 Crime Data')

In [ ]:
# CSA_DATA_Path_dict = bde.DownloadDataType('CSA_DATA')
# CSA_DATA_DF_dict = bde.GeoDataToDF(CSA_DATA_Path_dict)

In [ ]:
# CITY_DATA_Path_dict = bde.DownloadDataType('CITY_DATA')
# CITY_DATA_DF_dict = bde.GeoDataToDF(CITY_DATA_Path_dict)

# Cleaning "Part 1 Crime" Data Specifically

In [ ]:
from Baltimore_Data_Modules import Bmore_FeatureEngineering

crime_feat_dict = {'DROP':
                        ['RowID',
                         'Post',
                         'Gender',
                         'Age',
                         'Race',
                         'Ethnicity',
                         'GeoLocation',
                         'PremiseType',
                         'Total_Incidents',
                         'New_District'],
                   'CATEGORICAL':
                         ['CCNumber',
                          'CrimeCode',
                          'Description',
                          'Inside_Outside',
                          'Weapon',
                          'Location',
                          'Old_District',
                          'Neighborhood'],
                    'NUMERICAL':
                         []}

# instantiating feature engineering class
bcfe = Bmore_FeatureEngineering.BmoreCrimeFeatureEngineering(geo_cols_li=['Latitude',
                                                                          'Longitude'],
                                                                          time_col_str='CrimeDateTime',
                                                                          feat_col_dict=crime_feat_dict)
# Initial Cleaning of Baltimore crime data
Bmore_Crime_df = bcfe.CreateTimeIndexDF(FEAT_COLLECT_DF_dict['Part 1 Crime Data'])
Bmore_Crime_df = bcfe.DropUnwantedFeatsDF(Bmore_Crime_df)
Bmore_Crime_df = bcfe.GeoStatsDF(Bmore_Crime_df)
# filtering to Datetimes that make sense
Bmore_Crime_df = Bmore_Crime_df[Bmore_Crime_df.CrimeDateTime.astype(int) > 1000000000000] # 1600000000000
Bmore_Crime_df.index = pd.DatetimeIndex(list(Bmore_Crime_df.index))
print('Bmore Crime Shape:',Bmore_Crime_df.shape)

# Extracting categorical and periodic features
categorical_df = bcfe.CatFeatEncode(Bmore_Crime_df)
periodicity_df = bcfe.PeriodicityEncode(Bmore_Crime_df)
print('Categorical Feature Shape:',categorical_df.shape)
print('periodicity Feature Shape:',periodicity_df.shape)

# Merging features & scaling
FINAL_X_df = pd.concat([categorical_df,periodicity_df],axis=1)
print('Feature Shape (X):',FINAL_X_df.shape)
# Separating Prediction targets
FINAL_Y_df = Bmore_Crime_df[bcfe.geo_cols_].apply(pd.to_numeric)
print('Target Shape (Y):',FINAL_Y_df.shape)

# Combining everything together again
FINAL_DF = pd.concat([FINAL_X_df,FINAL_Y_df],axis=1).sort_index()
FINAL_DF = bcfe.DropZeroGeos(FINAL_DF)
print('Final Shape Regular (X+Y):',FINAL_DF.shape)

#resampling  for every hour
RULE_str = '1H' # 1H for every hour and 1D for every day
resampled_FINAL_DF = FINAL_DF.resample(rule=RULE_str).mean(numeric_only=True).interpolate(method='akima')
resampled_FINAL_shape = resampled_FINAL_DF.shape
print('Final Shape Resampled (X+Y):',resampled_FINAL_shape)

# Splitting Data Into Train, Validation, & Test Sets
- Normalizing data based on features in training set. Using quantile transformer

In [ ]:
import seaborn as sns

def DataSplitter(df):
    # Splitting training, validation, and test sets
    column_indices_dict = {name: i for i, name in enumerate(df.columns)}
    n = len(df)
    TRAIN_DF = df[0:int(n*0.7)]
    VAL_DF = df[int(n*0.7):int(n*0.9)]
    TEST_DF = df[int(n*0.9):]
    print('Train Shape:',TRAIN_DF.shape)
    print('Train Datetime Range:',
          str(TRAIN_DF.index.min())+' -> '+str(TRAIN_DF.index.max())+'\n')
    print('Validation Shape:',VAL_DF.shape)
    print('Validation Datetime Range:',
          str(VAL_DF.index.min())+' -> '+str(VAL_DF.index.max())+'\n')
    print('Test Shape:',TEST_DF.shape)
    print('Test Datetime Range:',
          str(TEST_DF.index.min())+' -> '+str(TEST_DF.index.max())+'\n')
    return {'TRAIN':TRAIN_DF,'VAL':VAL_DF,'TEST':TEST_DF}

split_dict = DataSplitter(resampled_FINAL_DF)

def NormalizeSplits(split_dict=split_dict):
    # Normalizing data sets via L2
    # Fitting to the training set
    norm = QuantileTransformer().fit(split_dict['TRAIN'])
    # Transforming all the split sets
    scaled_TRAIN_DF = pd.DataFrame(norm.transform(split_dict['TRAIN']),
                                   columns=norm.get_feature_names_out(),
                                   index=split_dict['TRAIN'].index)
    scaled_VAL_DF = pd.DataFrame(norm.transform(split_dict['VAL']),
                                   columns=norm.get_feature_names_out(),
                                   index=split_dict['VAL'].index)
    scaled_TEST_DF = pd.DataFrame(norm.transform(split_dict['TEST']),
                                    columns=norm.get_feature_names_out(),
                                    index=split_dict['TEST'].index)
    return norm, {'TRAIN':scaled_TRAIN_DF,'VAL':scaled_VAL_DF,'TEST':scaled_TEST_DF}

norm, scaled_split_dict = NormalizeSplits(split_dict)

# Creating violin plot to show data features
voilin_df = scaled_split_dict['TRAIN'].melt(var_name='Column', value_name='Normalized')
plt.figure(figsize=(12, 6))
ax = sns.violinplot(x='Column', y='Normalized', data=voilin_df)
_ = ax.set_xticklabels(list(scaled_split_dict['TRAIN'].columns), rotation=90)
plt.title('Training Data Features')
plt.show()

# Data Windowing

In [ ]:
from Baltimore_Data_Modules import Model_Utils

## Creating a multistep output to predict more than one window at a time

# OUT_STEPS = 7*2 # Each time step is 1 day. 7 days in a week
# INPUT_STEPS = 30*2 # Each time step is 1 day. About 30 days in a month
OUT_STEPS = 168*2 # Each time step is 1 hour. 168 hours in a week
INPUT_STEPS = 730*2 # Each time step is 1 hour. 730 hours in a month
num_features = resampled_FINAL_shape[1]

wg = Model_Utils.WindowGenerator(input_width=INPUT_STEPS,
                                 label_width=OUT_STEPS,
                                 label_columns=None, # Originally just latitude
                                 shift=OUT_STEPS,
                                 split_dict=scaled_split_dict)
wg

# Training LSTM

In [ ]:
MAX_EPOCHS = 20
multi_val_performance = {}
multi_performance = {}

def compile_and_fit(model, window, patience=2):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')
    # Created using M1/M2 Mac hence the use of "tf.keras.optimizers.legacy.Adam" instead of "tf.keras.optimizers.Adam"
    model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.legacy.Adam(), 
                metrics=[tf.keras.metrics.MeanAbsoluteError()])

    history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
    return history

multi_lstm_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units].
    # Adding more `lstm_units` just overfits more quickly.
    tf.keras.layers.LSTM(32, return_sequences=False),
    # Shape => [batch, out_steps*features].
    tf.keras.layers.Dense(OUT_STEPS*num_features,
                          kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features].
    tf.keras.layers.Reshape([OUT_STEPS, num_features])
])
train_start = datetime.now()
history = compile_and_fit(multi_lstm_model, wg)
IPython.display.clear_output()
train_finish = datetime.now() - train_start
print('Total Training Time {d}:{h}:{m}'.format(d=days_hours_minutes(train_finish)[0],
                                               h=days_hours_minutes(train_finish)[1],
                                               m=days_hours_minutes(train_finish)[2]))

multi_val_performance['LSTM'] = multi_lstm_model.evaluate(wg.val)
multi_performance['LSTM'] = multi_lstm_model.evaluate(wg.test, verbose=0)
multi_lstm_model.save(os.getcwd() + '/Bmore_Crime_Predict_Model.keras')

# Plotting Fit Examples From Train

In [ ]:
wg.plot(multi_lstm_model,plot_col=bcfe.geo_cols_[0])
wg.plot(multi_lstm_model,plot_col=bcfe.geo_cols_[1])

# LSTM Inference On Latest Data

In [ ]:
## Displaying metadata about model 
print('Input Shape:',multi_lstm_model.input_shape)
print('Output Shape:',multi_lstm_model.output_shape)
display(multi_lstm_model.summary())

## Getting the latest crime data and preparing for inference
deploy_df = wg.test_df.tail(INPUT_STEPS)
deploy_shape_tu = deploy_df.shape
deploy_ar = np.reshape(deploy_df.values,newshape=(1,deploy_shape_tu[0],deploy_shape_tu[1]))

## Obtaining forecasted datetime range
LATEST_DATETIME = deploy_df.tail(1).index[0]
FORECAST_DATE_RANGE = pd.date_range(start=LATEST_DATETIME,
                                    freq='1H',
                                    periods=OUT_STEPS+1)[1:]

## Generating inference and scaling back to original data
forecast_ar = multi_lstm_model.predict(deploy_ar)
forecast_df = pd.DataFrame(norm.inverse_transform(forecast_ar[0]),
                           columns=wg.column_indices,
                           index=FORECAST_DATE_RANGE)
geo_forecast_df = forecast_df[bcfe.geo_cols_]

# Visualizing Predicted Crime Coordinates

In [ ]:
import folium

geo_forecast_li = geo_forecast_df.values.tolist()

map_folium = folium.Map(location=[39.299236, -76.609383],
                  tiles="OpenStreetMap",
                  zoom_start=10)

for coord_index in range(len(geo_forecast_df)):
    pred_datetime_str = str(geo_forecast_df.iloc[coord_index].name)
    coordinates_li = geo_forecast_li[coord_index]
    map_folium.add_child(folium.Marker(location=coordinates_li,
                         popup='Prediction Datetime:<br>'
                         +pred_datetime_str))
map_folium

# Calculating different forms of error

In [ ]:
geo_sigma_se = FINAL_DF.describe().loc['std']
lat_error = geo_sigma_se[bcfe.geo_cols_[0]]
lon_error = geo_sigma_se[bcfe.geo_cols_[1]]
train_error = multi_val_performance['LSTM'][1]
test_error = multi_performance['LSTM'][1]

geo_uncertainty = math.sqrt((lat_error**2)+(lon_error**2))
model_error = math.sqrt((train_error**2)+(test_error**2))
print('Geo Uncertainty:',geo_uncertainty)
print('Model Error:',model_error)